<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h1>🚘 Automated Parking system</h1>
    <h3>- Intro </h3>
    <p> This notebook implements a system to get the optimum path to the nearest available parking bay in a parking lot for four vehicles. This would in turn help self driving vehicles to navigate the parking lot while getting the optimum parking spot for the driver. </p>
    <h3>- How to run </h3>
    <p>Chenge the value of the "count" parameter in the "spawn_parked_vehicles()" function in code cell 4 to any number from 1 - 22, then rerun all cells in the notebook to create new instances everytime.</p>
    <h3>Vehicle colors</h3>
    <ul>
        <li>Parked vehicles - orange </li>
        <li>Vehicle one - Blue</li>
        <li>Vehicle two - red</li>
        <li>Vehicle three - yellow</li>
        <li>Vehicle four - green</li>
    </ul>
    <h3>- Optional </h3>
    <p>Chenge the value of the "footprints" parameter in the "agent()" function in code cell 4 to True or False to toggle visible vehicle path travesal, then rerun all cells in the notebook to create new instances everytime.</p>
</div>

### Payamaze Source code changes:

The original source code for payamaze was forked and customized for use in this notebook hence, the offical source code version would give errors if used.

- Added the __'mazeName'__ attribute to the __'CreateMaze'__ method of maze class to give maze custom name when __'saveMaze'__ flag is set to True
- Added the __'windowName'__ attribute to the __'CreateMaze'__ method of maze class and is used by internal method __'drawMaze()'__ function which now serves as the TKinter GUI window title when simulator is started.
- Added color obects: 
    - parked_vehicle
    - current_vehicle
    - search_path

<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>📦 Package Imports</h2>
</div>

In [1]:
import os
import warnings
import random

# Run local instance of PYAMAZE package to access module exports
%run pyamaze_custom_source.ipynb

# config
warnings.filterwarnings(action='once')

<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>🐥 Function Declarations</h2>
    <p>Declaration of all functions used in the Notebook </p>
</div>

In [2]:
##############################################################################################################################
# 01 -> function to spawn parked vehicles                                                                                  ###
##############################################################################################################################

def spawn_parked_vehicles(mz, pointList, count = 5):
    """
    Spawns parked vehicles in random parking bays.
    
    Parameters:
    -----------
    mz -> Instance of pyamaze maze.
    pointList -> Marked cells representing parking bay locations.
    count (int) -> number of parked vehicles to spawn.
    
    Returns:
    --------
    spawn_points (set) -> Set containing locations of spawned parked vehicles.
    """
    # create empty set to hold parked vehicle spawn points
    spawn_points = set()
    
    # check that spawn count is less than spawn point list
    if count < len(pointList) - 3:
        # populate set with random unique values from the marked cells
        while len(spawn_points) < count:
            spawn_points.add(random.choice(pointList))
    
        # for every point, create an uncontrolled parked vehicle in the marked spot
        for point in spawn_points:
            agent(mz, shape = 'arrow', color = COLOR.parked_vehicle).position = point
    else:
        raise Exception(f'Count: {count}, is higher than number of available spawn points: ({len(pointList) - 4})') 
    
    # return spawn points that have been occupied
    return spawn_points


##############################################################################################################################
# 02 -> function to perform breadth first search                                                                           ###
##############################################################################################################################

def BFS(mz, goal):
    """
    Performs Breradth First Search (BFS) on a given maze
    
    Parameters:
    -----------
    mz -> maze instance of PyAmaze
    goal -> Cell position to be considered as goal/target
    
    returns:
    --------
    path_string (str) -> STring containing the optimal path to acheive the goal.
    
    """
    start = (mz.rows, mz.cols)
    # create 2 queues to hold the current stack and explored cells
    frontier = [start]
    explored = [start]
    
    bfs_path = {}
    
    while len(frontier) > 0:
        current_cell = frontier.pop(0)
        
        # break if the cell is the goal 
        if current_cell == goal:
            break
        
        for direction in 'ESNW':
            if mz.maze_map[current_cell][direction] == True:
                if direction == 'E':
                    child_cell = (current_cell[0], current_cell[1] + 1)
                elif direction == 'W':
                    child_cell = (current_cell[0], current_cell[1] - 1)
                elif direction == 'N':
                    child_cell = (current_cell[0] - 1, current_cell[1])
                elif direction == 'S':
                    child_cell = (current_cell[0] + 1, current_cell[1])
                    
                # do nothing if child cell is already explored 
                if child_cell in explored:
                    continue 
                    
                # otherwise append the child cell to the frontier and explored queues
                frontier.append(child_cell)
                explored.append(child_cell)
                
                # add this path to the bfs path dictionary
                bfs_path[child_cell] = current_cell
                
    fwd_path = {}
    cell = goal 
    
    while cell != start:
        fwd_path[bfs_path[cell]] = cell
        cell = bfs_path[cell]
        
    # NOTE:
    ##############################################################################################################################
    # using the fwd_path dictionatry as the path value for tracing worked but it gave a runtime exception in TKinter            ##
    # apparently for some reason, the goal cell is not being pushed to the dictionary but is being used in the trace function.  ##
    # Hence the key error.                                                                                                      ##
    # to solve this, the path dictionaries have been converted to directional strings as pyamaze accepts string sequence        ##
    # string sequence can be any of 'EWNS'                                                                                      ##
    ##############################################################################################################################
        
    # generate path string from path dictionary 
    path_string = ''
    for key, value in fwd_path.items():
        # Rule 1
        if value[1] == key[1] - 1:
            path_string += 'W'
            
        # Rule 2
        if value[1] == key[1] + 1:
            path_string += 'E'
            
        # Rule 4
        if value[0] == key[0] + 1:
            path_string += 'S'
            
        # Rule 4
        if value[0] == key[0] - 1:
            path_string += 'N'
            
    # return reversed string as path string is created from goal to starting point
    return path_string[::-1]
                

<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>🧱 Create Environment</h2>
    <ul>
        <li>Create pyamaze environment to simulate car park </li>
        <li>Add parked vehicles in environment </li>
        <li>Set goal for ego vehicle</li>
    </ul>
</div>

In [3]:
# initialize maze 
parking_lot = maze(10, 14)

# initialize parking goal 
parking_goal = (1, 1)

# create maze
# This was run once to create a local maze file. 
# parking_lot.CreateMaze(mazeName = 'parking_lot_maze', windowName = 'New test Window name', pattern = 'h')

# load maze 
parking_lot.CreateMaze(loadMaze = 'parking_lot_maze.csv', windowName = 'EMMANUEL AI FOUNDATION - Q2002311')

# set parking spots based on the map
parking_lot.markCells = [(3,2), (4,2), (5,2), (6,2), (7,2), (8,2), (9, 2),
                         (3,7), (4,7), (5,7), (6,7), (7,7), (8,7),
                         (3,8), (4,8), (5,8), (6,8), (7,8), (8,8),
                         (3,13), (4,13), (5,13), (6,13), (7,13), (8,13), (9, 13)]


Path to goal not found!


<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>🏆 Create Agents (Ego vehicle & Parked vehciles)</h2>
    <ul>
        <li>Create ego vehicles (vehicles to be parked)</li>
        <li>Randomly pawn parked vehicles in the parking lot</li>
        <li>Get spawn points of bays that have been occupied by parked vehicles</li>
    </ul>
</div>

In [4]:
# create ego vehicle agent
vehicle_one = agent(parking_lot, shape = 'arrow', color = COLOR.vehicle_one, footprints = True)
vehicle_two = agent(parking_lot, shape = 'arrow', color = 'red', footprints = True)
vehicle_three = agent(parking_lot, shape = 'arrow', color = 'yellow', footprints = True)
vehicle_four = agent(parking_lot, shape = 'arrow', color = 'green', footprints = True)

# get the spawn points of the parked vehicles
used_bays = spawn_parked_vehicles(count = 20, pointList = parking_lot.markCells, mz = parking_lot)

# view used bays 
used_bays

{(3, 2),
 (3, 7),
 (3, 8),
 (3, 13),
 (4, 2),
 (4, 8),
 (4, 13),
 (5, 7),
 (5, 8),
 (5, 13),
 (6, 13),
 (7, 2),
 (7, 7),
 (7, 8),
 (7, 13),
 (8, 2),
 (8, 7),
 (8, 8),
 (8, 13),
 (9, 2)}

<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>🤲 What's left?</h2>
    <ul>
        <li>Get the spawn points that remain available</li>
    </ul>
</div>

In [5]:
# create set for available bays 
available_bays = set()

# get acvailable spawn points 
for point in parking_lot.markCells:
    if point not in used_bays:
        available_bays.add(point)

# check how many bays are availble
len(available_bays)


6

<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>🏆 Perform Breadth First Search (BFS) on the available spawn points</h2>
    <ul>
        <li>Loop through all spawn points</li>
        <li>create a dictionary of spawn point, path and length of path</li>
        <li>Get the four closest spawn points</li>
    </ul>
</div>

In [6]:

path_lengths = []

# Find the closest bay 
for bay in available_bays:
    path = BFS(mz = parking_lot, goal = bay)
    path_lengths.append({
        'spawn_point': bay,
        'path': path, 
        'path_len': len(path)
    })


# sort path length for available bays 
sorted_path_lengths = sorted(path_lengths, key = lambda x: x['path_len'])

# get the 4 closest bays
closest_bays = sorted_path_lengths[:4]

# show bay information
for index, bay in enumerate(closest_bays):
    print('{}. spawn point -> {} | path -> {}'.format(index + 1, bay['spawn_point'], bay['path']))

1. spawn point -> (9, 13) | path -> WWWNEE
2. spawn point -> (6, 8) | path -> WWWNWNNNWW
3. spawn point -> (6, 7) | path -> WWWNWWWWWWNNNEE
4. spawn point -> (6, 2) | path -> WWWNWWWWWWWNNNWW


<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>✨ Move the vehicles to the nearest optimal available bay</h2>
    <ul>
        <li>Create ego vehicles (vehicles to be parked)</li>
        <li>Randomly pawn parked vehicles in the parking lot</li>
        <li>Get spawn points of bays that have been occupied by parked vehicles</li>
    </ul>
</div>

In [7]:
# move the four vehicles to the closest available bays 
parking_lot.tracePath({vehicle_one: closest_bays[0]['path']}, delay = 200)
parking_lot.tracePath({vehicle_two: closest_bays[1]['path']}, delay = 200)
parking_lot.tracePath({vehicle_three: closest_bays[2]['path']}, delay = 200)
parking_lot.tracePath({vehicle_four: closest_bays[3]['path']}, delay = 200)


# show each vehicles bay on the simulator window
textLabel(parking_lot, 'vehicle 1 cells moved', closest_bays[0]['path_len'])
textLabel(parking_lot, 'vehicle 2 cells moved', closest_bays[1]['path_len'])
textLabel(parking_lot, 'vehicle 3 cells moved', closest_bays[2]['path_len'])
textLabel(parking_lot, 'vehicle 4 cells moved', closest_bays[3]['path_len'])

<div style = 'font-size: 16px; color: white; background-color: #2F2E3E; padding: 15px 10px; border-radius: 4px; border-radius: 10px'>
    <h2>🏃 Run Simulation</h2>
</div>

In [8]:
parking_lot.run()

View code on github -> [Github Link](https://github.com/oddFEELING/automated-car-park)